In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import os
import math
from getpass import getuser
pd.set_option('display.max_columns', None)

In [2]:
rutas_por_usuario = {
    "antmiguel": {
        "rawData": os.path.join(os.path.pardir, 'data', 'raw'),
        "externalData": os.path.join(os.path.pardir, 'data', 'external'),
        "interimData": os.path.join(os.path.pardir, 'data', 'interim'),
        "processedData": os.path.join(os.path.pardir, 'data', 'processed')
    },
    "jupyter": {
        'rawData': 'gs://ent-prd-sandbox-fdo-bucket/ent-prd-sandbox-fdo-bucket/ScoreCI/raw/',
        'externalData': 'gs://ent-prd-sandbox-fdo-bucket/ent-prd-sandbox-fdo-bucket/ScoreCI/external/',
        'interimData': 'gs://ent-prd-sandbox-fdo-bucket/ent-prd-sandbox-fdo-bucket/ScoreCI/interim/',
        'processedData': 'gs://ent-prd-sandbox-fdo-bucket/ent-prd-sandbox-fdo-bucket/ScoreCI/processed/'
    }
}

usuario = getuser()
rutas = rutas_por_usuario.get(usuario, {})

rutas

{'rawData': '..\\data\\raw',
 'externalData': '..\\data\\external',
 'interimData': '..\\data\\interim',
 'processedData': '..\\data\\processed'}

In [3]:
dfCalendar = pd.read_parquet(os.path.join(rutas.get('externalData'),'isoCalendar.parquet'))

In [4]:
dfHist = pd.read_parquet(os.path.join(rutas.get('rawData'), 'baseAnaliticaHistorico_23092024_abr24v5.parquet'))
dfCliente = pd.read_parquet(os.path.join(rutas.get('rawData'), 'baseAnaliticaCliente_23092024_abr24v5.parquet'))
dfAsesor = pd.read_parquet(os.path.join(rutas.get('rawData'), 'baseAnaliticaAsesor_23092024_abr24v5.parquet'))

In [5]:
dfHist['CORTE'] = pd.to_datetime(dfHist['CORTE'], errors='coerce')

In [6]:
dfHist = pd.merge(dfHist, dfCalendar, left_on='CORTE', right_on='fecha', how = 'inner')
dfHist.drop(columns=['fecha'], inplace=True)

In [7]:
dfHist.head()

,EMPLOYEE_ID,EMPLOYEE_PAYROLL_NUM,CARTERA_ASESOR,CR1_7,CR8_8_15,CR16_60,CRMAS_60,SINATRASO,CORTE,diaSemana,semana,mesCalendario,nombreMes,dia,tipoDia,cosecha
0,48405724,95955,2074311.40,0.0,0.0,0.0,0.0,2074311.40,2023-01-31,2,5,1,January,31,1,202301
1,74087870,110936,407116.69,0.0,0.0,0.0,0.0,407116.69,2023-01-31,2,5,1,January,31,1,202301
2,72532463,105914,1212145.04,0.0,0.0,0.0,0.0,1212145.04,2023-01-31,2,5,1,January,31,1,202301
3,73608661,110438,1034583.84,0.0,0.0,0.0,0.0,1034583.84,2023-01-31,2,5,1,January,31,1,202301
4,66027319,102560,947052.13,0.0,0.0,0.0,0.0,947052.13,2023-01-31,2,5,1,January,31,1,202301


In [13]:
pd.pivot_table(
    data=dfHist,
    values='CARTERA_ASESOR',
    index='EMPLOYEE_ID',
    columns='cosecha',
    aggfunc='sum',
    fill_value=0
).reset_index()

cosecha,EMPLOYEE_ID,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,1949827,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,305943.42,799631.39,1167942.81,1697317.15,1848221.54,2347603.55,2444090.63,2422892.18,2397582.20,2543739.38,2986016.65,3719906.66,3896374.22,4301060.01,3978351.11
1,1949838,7707009.1,7936044.4,8235858.59,8416221.55,8713555.42,7075754.44,7238483.15,7681158.17,7912070.54,8077168.70,8544405.89,8757362.33,8958436.80,8905744.01,9437116.37,9492756.47,9796621.33,9960364.91,9918111.58,9777707.11,7985848.38,8200772.86,8785037.87,9580403.28,9915713.20
2,1949853,0.0,0.0,0.00,0.00,210521.37,509497.04,707254.49,1097432.53,1693461.94,2286600.07,2565855.35,2779391.79,3012884.11,3616313.09,4110999.98,4446808.59,4581352.98,5440169.49,5436539.36,6130085.65,6257454.06,6472648.75,6427286.44,6481714.03,6103116.58
3,1950001,0.0,0.0,0.00,0.00,0.00,0.00,0.00,1207215.89,1294111.71,3138583.15,3669365.34,3577158.25,5137568.63,5548379.50,5390391.56,5778758.60,5772974.38,5804871.09,5988514.96,5996740.00,6518747.22,6285878.86,6115246.96,5927624.36,6033793.98
4,1950569,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4589406.22,4921139.30,6642370.30,6767213.33,8455821.07,8799084.84,8926327.91,9248610.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,88475663,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,255999.79,1869253.86,2319137.28,3689468.24
1904,88475673,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4044705.36,4100286.97,4211409.04,4441850.34
1905,88597493,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5427876.34,269499.08,119769.07,0.00
1906,88695801,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6780858.54,5810344.47,6006055.83,1594163.35
